In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
from subprocess import check_call
from glob import glob

import socket

import yaml
import textwrap

import cesm_tools

project_name = 'mom6-marbl-cases'
this_notebook = 'create-new-case.ipynb'

assert 'cheyenne' in socket.gethostname(), (
    'this notebook must be run on cheyenne login node'
)

## Set up local directory and machine-dependent paths

In [3]:
USER = os.environ["USER"]
scriptroot = os.getcwd()

machine = "cheyenne"
if machine in ["cheyenne"]:
    inputdata = "/glade/p/cesmdata/cseg/inputdata"
    scratch = f"/glade/scratch/{USER}"
    project_root = f'/glade/work/{USER}/{project_name}'
    project_data = f'{project_root}/data'
    caseroot_root = f'{project_root}/cases'
else:
    raise ValueError(f"unknown machine: {machine}")
    
coderoot = f"{project_root}/codes"

## Check out model code and configure sandbox 

In [4]:
remote = 'git@github.com:mnlevy1981/cesm.git'
cesmtag = 'cesm2_3_beta08+mom6_marbl'

cesmroot = cesm_tools.code_checkout(remote, coderoot, cesmtag)

b"Cloning into 'cesm2_3_beta08+mom6_marbl'...\n"
Branch 'cesm2_3_beta08+mom6_marbl' set up to track remote branch 'cesm2_3_beta08+mom6_marbl' from 'origin'.


Switched to a new branch 'cesm2_3_beta08+mom6_marbl'


Processing externals description file : Externals.cfg
Checking status of externals: ccs_config, cice, cice6, cmeps, cdeps, cpl7, share, mct, parallelio, cime, fms, mom, pop, ww3, ww3dev, 
Checking out externals: 

e-  ./ccs_config
        -, not checked out --> ccs_config_cesm0.0.16
    git clone --quiet https://github.com/ESMCI/ccs_config_cesm.git ccs_config
    git checkout --quiet ccs_config_cesm0.0.16

e-  ./cime
        -, not checked out --> cime6.0.15
    git clone --quiet https://github.com/ESMCI/cime cime
    git checkout --quiet cime6.0.15

e-  ./components/cdeps
        -, not checked out --> cdeps0.12.41
    git clone --quiet https://github.com/ESCOMP/CDEPS.git cdeps
    git checkout --quiet cdeps0.12.41
Processing externals description file : Externals_CDEPS.cfg
Checking out externals: 

--- 
        -,  --> 
    git clone --quiet https://github.com/ESMCI/fox.git fox
    git checkout --quiet 4.1.2.1

--- 
        -,  --> 
    git clone --quiet https://github.com/PARALLELIO

## Setup case options

Information on compsets is [here](https://www.cesm.ucar.edu/models/cesm2/config/2.1.3/compsets.html).

In [5]:
clobber = True

project = "P93300670"

member_id = 1
compset = 'C1850MOMMARBL'
resolution = "TL319_t061"

description = """
my first MOM6 run
"""

case = f'{compset}.{resolution}.{member_id:03d}'

# run setting
resubmit = 0
stop_n = 1
stop_option = "nmonth"
rest_n = 1
rest_option = "nyear"
wallclock = "12:00:00"
queue = "regular"

# paths
caseinfo_file = f"case-info/{case}.yml"
caseroot = f"{caseroot_root}/{case}"
rundir = f"{scratch}/{case}"
archive_root = f"{scratch}/archive/{case}"

## Create case

### Determine if existing case exists, remove if `clobber=True`.

In [6]:
# ensure root directories
os.makedirs(os.path.dirname(caseroot_root), exist_ok=True)
os.makedirs(os.path.dirname(caseinfo_file), exist_ok=True)

if clobber:
    check_call(['rm', '-f', caseinfo_file])

if os.path.exists(caseinfo_file):
    raise ValueError(f'case {case} exists\nset clobber=True to remove')


with open(caseinfo_file, "w") as fid:
    yaml.dump(
        dict(
            case=case,
            description=description,
            caseroot=caseroot,
            rundir=rundir,
            archive_root=archive_root,
            compset=compset,
            resolution=resolution,
        ),
        fid,
    )
if os.path.exists(caseroot):
    print(f"deleting {caseroot}")
    check_call(['rm', '-fr', caseroot])
    
if os.path.exists(rundir):
    print(f"deleting {rundir}")
    check_call(['rm', '-fr', rundir])

In [7]:
check_call(
    [
        "./create_newcase",
        "--case",
        caseroot,
        "--project",
        project,
        "--machine",
        machine,
        "--res",
        resolution,
        "--compset",
        compset,
        "--run-unsupport",
    ],
    cwd=f"{cesmroot}/cime/scripts",
)

Compset longname is 1850_DATM%NYF_SLND_DICE%SSMI_MOM6%MARBL_DROF%NYF_SGLC_SWAV
Compset specification file is /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/mom//cime_config/config_compsets.xml
Automatically adding SESP to compset
Compset forcing is 1850
ATM component is  Data driven ATM COREv2 normal year forcing
LND component is Stub land component
ICE component is dice mode is ssmi
OCN component is MOM6 OCEAN MARBL Biogeochemistry Tracer Library
ROF component is Data runoff modelCOREv2 normal year forcing:
GLC component is Stub glacier (land ice) component
WAV component is Stub wave component
ESP component is Stub external system processing (ESP) component
Pes     specification file is /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/mom//cime_config/config_pes.xml
Machine is cheyenne
Pes setting: grid match    is a%TL319.+oi%tx0.66v1 
Pes setting: machine match is cheyenne 
Pes setting: compset_match is _DATM.+_DICE.*_MOM

Variable MAX_GPUS_PER_NODE not defined for machine cheyenne


 Compset is: 1850_DATM%NYF_SLND_DICE%SSMI_MOM6%MARBL_DROF%NYF_SGLC_SWAV_SESP 
 Grid is: a%TL319_l%null_oi%tx0.66v1_r%JRA025_g%null_w%null_z%null_m%tx0.66v1 
 Components in compset are: ['datm', 'slnd', 'dice', 'mom', 'drof', 'sglc', 'swav', 'sesp'] 
No charge_account info available, using value from PROJECT
cesm model version found: cesm2_3_alpha07b-34-g1b0fff8
Batch_system_type is pbs
job is case.run USER_REQUESTED_WALLTIME None USER_REQUESTED_QUEUE None WALLTIME_FORMAT %H:%M:%S
job is case.st_archive USER_REQUESTED_WALLTIME None USER_REQUESTED_QUEUE None WALLTIME_FORMAT %H:%M:%S
 Creating Case directory /glade/work/mclong/mom6-marbl-cases/cases/C1850MOMMARBL.TL319_t061.001


0

## Configure case

### Apply XML settings

Documentation is provided [here](https://www.cesm.ucar.edu/models/cesm2/settings/2.1.3/).

In [8]:
def xmlchange(arg):
    """call xmlchange"""
    check_call(["./xmlchange", arg], cwd=caseroot)

In [9]:
# initialization
xmlchange("RUN_TYPE=startup")
xmlchange("OCN_BGC_HIST_VERT_GRID=native")

xmlchange(f"RESUBMIT={resubmit}")
xmlchange(f"STOP_OPTION={stop_option}")
xmlchange(f"STOP_N={stop_n}")
xmlchange(f"REST_OPTION={rest_option}")
xmlchange(f"REST_N={rest_n}")
xmlchange(f"JOB_WALLCLOCK_TIME={wallclock}")
xmlchange(f"JOB_QUEUE={queue}");

## Run setup

In [10]:
check_call(['./case.setup'], cwd=caseroot)

Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
job is case.run USER_REQUESTED_WALLTIME 12:00:00 USER_REQUESTED_QUEUE regular WALLTIME_FORMAT %H:%M:%S
Creating batch scripts
Writing case.run script from input template /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/ccs_config/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/ccs_config/machines/template.st_archive
Creating file case.st_archive
Creating user_nl_xxx files for components and cpl
If an old case build already exists, might want to run 'case.build --clean' before building
You can now run './preview_run' to get more info on how your case will be run


0

## SourceMods

Copy source mods to `CASEROOT`

In [11]:
src_mods_files = {
    'mom': glob(f"{scriptroot}/SourceMods/src.mom/*"),
}
for component, files in src_mods_files.items():
    for src in files:
        src_basename = os.path.basename(src)
        dst = f"{caseroot}/SourceMods/src.{component}/{src_basename}"
        shutil.copyfile(src, dst)

## Namelist settings

### Write namelist modifcations

Documentation for the POP namelist is [here](https://www.cesm.ucar.edu/models/cesm2/settings/2.1.3/pop2_nml.html).

In [12]:
def list_to_nml_list(lst):
    return ",".join([f"'{v}'" for v in lst])

user_nl = dict()

for key, nl in user_nl.items():
    with open(f"{caseroot}/user_nl_{key}", 'w') as fid:
        fid.write(user_nl[key])

## Build the model

In [13]:
check_call(['qcmd', '-A', project, '--', './case.build'], cwd=caseroot)

Submitting command to PBS using account P93300670:
    ./case.build

Requested custom PBS options:
    -A P93300670 

Waiting for job 6302902.chadmin1.ib0.cheyenne.ucar.edu to start ... 


Building case in directory /glade/work/mclong/mom6-marbl-cases/cases/C1850MOMMARBL.TL319_t061.001
sharedlib_only is False
model_only is False
Setting resource.RLIMIT_STACK to -1 from (-1, -1)
Generating component namelists as part of build
  2022-09-09 09:03:36 atm 
Create namelist for component datm
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/cdeps/datm/cime_config/buildnml
  2022-09-09 09:03:36 lnd 
Create namelist for component slnd
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/cime/src/components/stub_comps_nuopc/slnd/cime_config/buildnml
  2022-09-09 09:03:36 ice 
Create namelist for component dice
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/cdeps/dice/cime_config/buildnml

0

## Preview namelists

In [14]:
check_call(['./preview_namelists'], cwd=caseroot)

Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
  2022-09-09 09:21:20 atm 
Create namelist for component datm
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/cdeps/datm/cime_config/buildnml
  2022-09-09 09:21:20 lnd 
Create namelist for component slnd
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/cime/src/components/stub_comps_nuopc/slnd/cime_config/buildnml
  2022-09-09 09:21:20 ice 
Create namelist for component dice
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/cdeps/dice/cime_config/buildnml
  2022-09-09 09:21:20 ocn 
Create namelist for component mom
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/mom//cime_config/buildnml
  2022-09-09 09:21:21 rof 
Create namelist for component drof
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/cdeps/drof/cime_config/buildnml
  2022-09-

0

## Submit the job

In [15]:
check_call(['./case.submit'], cwd=caseroot)


env_batch.xml appears to have changed, regenerating batch scripts
manual edits to these file will be lost!



Creating batch scripts
Writing case.run script from input template /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/ccs_config/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/ccs_config/machines/template.st_archive
Creating file case.st_archive
Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
  2022-09-09 09:21:23 atm 
Create namelist for component datm
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/cdeps/datm/cime_config/buildnml
  2022-09-09 09:21:23 lnd 
Create namelist for component slnd
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/cime/src/components/stub_comps_nuopc/slnd/cime_config/buildnml
  2022-09-09 09:21:23 ice 
Create namelist for component dice
   Calling /glade/work/mclong/mom6-marbl-cases/codes/cesm2_3_beta08+mom6_marbl/components/cdeps/dice/cime_c

submit_jobs case.run
Submit job case.run


Submitted job id is 6302957.chadmin1.ib0.cheyenne.ucar.edu
Submitting job script qsub -q regular -l walltime=12:00:00 -A P93300670  -W depend=afterok:6302957.chadmin1.ib0.cheyenne.ucar.edu -m ea -v ARGS_FOR_SCRIPT='--resubmit' case.st_archive
Submitted job id is 6302958.chadmin1.ib0.cheyenne.ucar.edu
Submitted job case.run with id 6302957.chadmin1.ib0.cheyenne.ucar.edu
Submitted job case.st_archive with id 6302958.chadmin1.ib0.cheyenne.ucar.edu


Submit job case.st_archive


0

### Copy this notebook to create record of case creation

In [16]:
%notebook -e {case}.ipynb